In [45]:
faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_original/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_pair_evaluation/FAUST_r/off'

In [ ]:
import trimesh

scene = trimesh.Scene()

In [ ]:
# read tr_reg_000.off from each folder and plot it
# load the mesh from the original folder
scene.geometry.clear()

mesh_orig = trimesh.load(faust_orig_folder + '/tr_reg_099.off', process=False)
scene.add_geometry(mesh_orig)

# load the mesh from the remeshed folder
mesh_remeshed = trimesh.load(faust_remeshed_folder + '/tr_reg_099.off', process=False)
mesh_remeshed.vertices += [1, 0, 0]  # move the remeshed mesh to the right
scene.add_geometry(mesh_remeshed)

print(mesh_orig)
print(mesh_remeshed)

scene.show()

# SMPL correspondences on remeshed

In [ ]:
import my_code.datasets.preprocessing as preprocessing
import torch

verts_orig = preprocessing.center(
    torch.tensor(mesh_orig.vertices)
    )[0]

verts_remeshed = preprocessing.center(
    torch.tensor(mesh_remeshed.vertices)
    )[0]

verts_remeshed = preprocessing.scale(
    input_verts=verts_remeshed,
    input_faces=torch.tensor(mesh_remeshed.faces),
    ref_verts=verts_orig,
    ref_faces=torch.tensor(mesh_orig.faces)
)[0]

In [ ]:
# plot the meshes

scene.geometry.clear()

scene.add_geometry(trimesh.Trimesh(vertices=verts_orig, faces=mesh_orig.faces))
scene.add_geometry(trimesh.Trimesh(vertices=verts_remeshed + torch.tensor([0.0, 0, 0]),
                                   faces=mesh_remeshed.faces))

scene.show()

In [ ]:
from utils.fmap_util import nn_query

# find correspondences by nearest neigbor search

p2p_o_r = nn_query(
    verts_remeshed,
    verts_orig,   
)

print(p2p_o_r.shape)

In [ ]:
scene.geometry.clear()

# cmap = trimesh.visual.color.interpolate(data_x['verts'][:, 1], 'jet')
# imterpolate the color diagonally based on 0 and 2 coordinate
cmap = trimesh.visual.color.interpolate(
    verts_remeshed[:, 0] / verts_remeshed[:, 0].max() +\
    verts_remeshed[:, 1] / verts_remeshed[:, 1].max(),
    'jet')

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=verts_remeshed, faces=mesh_remeshed.faces)
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)].clip(0, 255)
scene.add_geometry(mesh1)

mesh2 = trimesh.Trimesh(vertices=verts_orig + torch.tensor([1, 0, 0]), faces=mesh_orig.faces)
cmap2 = cmap[p2p_o_r]
mesh2.visual.vertex_colors = cmap2.clip(0, 255)[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

In [ ]:
cmap[corr_r_o].shape

In [ ]:
import my_code.datasets.preprocessing as preprocessing
import torch
import trimesh
import os
import numpy as np
from tqdm import tqdm
from utils.fmap_util import nn_query
import shutil

faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_original/off'

faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/off'
corr_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/corres'

shutil.rmtree(corr_remeshed_folder, ignore_errors=True)
# shutil.rmtree(faust_remeshed_folder, ignore_errors=True)

os.makedirs(corr_remeshed_folder, exist_ok=True)
# os.makedirs(faust_remeshed_folder, exist_ok=True)
    

for file_name in tqdm(sorted(os.listdir(faust_remeshed_folder)), total=len(os.listdir(faust_remeshed_folder))):
    
    mesh_orig = trimesh.load(faust_orig_folder + f'/{file_name}', process=False)
    mesh_remeshed = trimesh.load(faust_remeshed_folder + f'/{file_name}', process=False)

    verts_orig = preprocessing.center(
        torch.tensor(mesh_orig.vertices)
        )[0]

    verts_remeshed = preprocessing.center(
        torch.tensor(mesh_remeshed.vertices)
        )[0]

    verts_remeshed = preprocessing.scale(
        input_verts=verts_remeshed,
        input_faces=torch.tensor(mesh_remeshed.faces),
        ref_verts=verts_orig,
        ref_faces=torch.tensor(mesh_orig.faces)
    )[0]
   
    p2p_o_r = nn_query(
    verts_remeshed,
    verts_orig,
    )
    p2p_o_r += 1
    
    
    # assert corr_r_o.shape[0] == verts_remeshed.shape[0]
    assert p2p_o_r.shape[0] == verts_orig.shape[0]
    assert p2p_o_r.min() > 0

    # save corr_r_o to corres folder, file_name .vts
    with open(corr_remeshed_folder + f'/{file_name[:-4]}.vts', 'w') as f:
        np.savetxt(f, p2p_o_r.numpy(), fmt='%d')


In [ ]:
import numpy as np
import torch


faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_original/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/off'
p2p_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/corres'


# read tr_reg_000.off from each folder and plot it
# load the mesh from the original folder
scene.geometry.clear()

# add axis
scene.add_geometry(trimesh.creation.axis(axis_length=1))

mesh_name = 'tr_reg_081'

mesh_orig = trimesh.load(faust_orig_folder + f'/{mesh_name}.off', process=False)

# load the mesh from the remeshed folder
mesh_remeshed = trimesh.load(faust_remeshed_folder + f'/{mesh_name}.off', process=False)
mesh_remeshed.vertices += [0.1, 0, 0]  # move the remeshed mesh to the right

p2p = np.loadtxt(p2p_folder + f'/{mesh_name}.vts')
p2p -= 1


cmap = trimesh.visual.color.interpolate(
    mesh_remeshed.vertices[:, 0] / mesh_remeshed.vertices[:, 0].max() +\
    mesh_remeshed.vertices[:, 1] / mesh_remeshed.vertices[:, 1].max(),
    'jet')

mesh_remeshed.visual.vertex_colors = cmap[:len(mesh_remeshed.vertices)].clip(0, 255)
scene.add_geometry(mesh_remeshed)


mesh2 = trimesh.Trimesh(vertices=mesh_orig.vertices + torch.tensor([1, 0, 0]).numpy(), faces=mesh_orig.faces)
cmap2 = cmap[p2p.astype(int)]
mesh2.visual.vertex_colors = cmap2.clip(0, 255)[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

# Scale FAUST original for pair evaluation

In [92]:
import my_code.datasets.preprocessing as preprocessing
import torch
import trimesh
import os
import numpy as np
from tqdm import tqdm
from utils.fmap_util import nn_query
import shutil

faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_original/off'
faust_new_folder = '/home/s94zalek_hpc/shape_matching/data_pair_evaluation/FAUST_original_scaled/off'

faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_r/off'
# corr_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_a/corres'

# shutil.rmtree(corr_remeshed_folder, ignore_errors=True)
# shutil.rmtree(faust_remeshed_folder, ignore_errors=True)
shutil.rmtree(faust_new_folder, ignore_errors=True)

# os.makedirs(corr_remeshed_folder, exist_ok=True)
# os.makedirs(faust_remeshed_folder, exist_ok=True)
os.makedirs(faust_new_folder, exist_ok=True)
    

for file_name in tqdm(sorted(os.listdir(faust_remeshed_folder)), total=len(os.listdir(faust_remeshed_folder))):
    
    mesh_orig = trimesh.load(faust_orig_folder + f'/{file_name}', process=False)
    mesh_remeshed = trimesh.load(faust_remeshed_folder + f'/{file_name}', process=False)

    verts_remeshed = preprocessing.center(
        torch.tensor(mesh_remeshed.vertices)
        )[0]

    verts_orig = preprocessing.center(
        torch.tensor(mesh_orig.vertices)
        )[0]
    
    verts_orig = preprocessing.scale(
        input_verts=verts_orig,
        input_faces=torch.tensor(mesh_orig.faces),
        ref_verts=verts_remeshed,
        ref_faces=torch.tensor(mesh_remeshed.faces)
    )[0]
    
    # center again
    # verts_orig = preprocessing.center(verts_orig)[0]

    # save original mesh 
    new_mesh_orig = trimesh.Trimesh(vertices=verts_orig, faces=mesh_orig.faces)
    
    new_mesh_orig.export(faust_new_folder + f'/{file_name}')
    
    # if file_name == 'tr_reg_011.off':
    #     break
    
# scene.geometry.clear()

# mesh_remeshed.vertices += [0.0, 0, 0]  # move the remeshed mesh to the right
# mesh_orig.vertices += [2, 0, 0]  # move the remeshed mesh to the right

# # add axis
# scene.add_geometry(trimesh.creation.axis(axis_length=1))

# scene.add_geometry(new_mesh_orig)
# scene.add_geometry(mesh_remeshed)
# scene.add_geometry(mesh_orig)


# scene.show()
    
    
    
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.91it/s]


In [ ]:
import trimesh
scene = trimesh.Scene()

In [ ]:


faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data_pair_evaluation/FAUST_original_scaled/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_pair_evaluation/FAUST_r/off'

shape_name = 'tr_reg_026'
mesh_orig_scaled = trimesh.load(faust_orig_folder + f'/{shape_name}.off', process=False)
mesh_orig = trimesh.load(f'/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_original/off/{shape_name}.off', process=False)
mesh_remeshed = trimesh.load(f'/home/s94zalek_hpc/shape_matching/data_pair_evaluation/FAUST_r/off/{shape_name}.off', process=False)

mesh_orig.vertices += [1, 0, 0]
# mesh_remeshed.vertices += [2, 0, 0]


scene.geometry.clear()

scene.add_geometry(mesh_orig_scaled)
scene.add_geometry(mesh_orig)
scene.add_geometry(mesh_remeshed)

scene.show()


In [37]:
# read /home/s94zalek_hpc/SGA18_orientation_BCICP_dataset/Dataset/FAUST/vtx_5k/corres/sampleID.vts

import numpy as np

sga_vts = np.loadtxt('/home/s94zalek_hpc/SGA18_orientation_BCICP_dataset/Dataset/FAUST/vtx_5k/corres/sampleID.vts')

sga_vts.shape

(5000,)

In [38]:
sga_vts.min(), sga_vts.max()

(1.0, 6888.0)

In [48]:
faust_orig_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_original/off'
faust_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data/FAUST_r/'

In [50]:
mesh_orig = trimesh.load(faust_orig_folder + '/tr_reg_099.off', process=False)
# scene.add_geometry(mesh_orig)

# load the mesh from the remeshed folder
mesh_remeshed = trimesh.load(faust_remeshed_folder + 'off/tr_reg_099.off', process=False)
# mesh_remeshed.vertices += [1, 0, 0]  # move the remeshed mesh to the right

verts_orig = torch.tensor(mesh_orig.vertices)
verts_remeshed = torch.tensor(mesh_remeshed.vertices)

mesh_orig, mesh_remeshed

(<trimesh.Trimesh(vertices.shape=(6890, 3), faces.shape=(13776, 3), name=`tr_reg_099.off`)>,
 <trimesh.Trimesh(vertices.shape=(5000, 3), faces.shape=(9996, 3), name=`tr_reg_099.off`)>)

# Check sampleID.vts file

In [59]:
template_smpl_corr = np.loadtxt('/home/s94zalek_hpc/SGA18_orientation_BCICP_dataset/Dataset/FAUST/vtx_5k/corres/sampleID.vts').astype(int)
template_smpl_corr -= 1

shape_template_corr = np.loadtxt(faust_remeshed_folder + 'corres/tr_reg_099.vts').astype(int)
shape_template_corr -= 1

In [64]:
shape_smpl_corr = template_smpl_corr[shape_template_corr]

In [57]:
shape_smpl_corr

array([6713, 1782, 3249, ..., 1352, 6550, 6636])

In [ ]:
scene.geometry.clear()

# cmap = trimesh.visual.color.interpolate(data_x['verts'][:, 1], 'jet')
# imterpolate the color diagonally based on 0 and 2 coordinate
cmap = trimesh.visual.color.interpolate(
    verts_orig[:, 0] / verts_orig[:, 0].max() +\
    verts_orig[:, 1] / verts_orig[:, 1].max(),
    'jet')

# add the first mesh
mesh1 = trimesh.Trimesh(vertices=verts_orig, faces=mesh_orig.faces)
mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)].clip(0, 255)
scene.add_geometry(mesh1)

mesh2 = trimesh.Trimesh(vertices=verts_remeshed + torch.tensor([1, 0, 0]), faces=mesh_remeshed.faces)
# cmap2 = cmap[shape_smpl_corr]

cmap2 = trimesh.visual.color.interpolate(
    verts_remeshed[:, 0] / verts_remeshed[:, 0].max() +\
    verts_remeshed[:, 1] / verts_remeshed[:, 1].max(),
    'jet')

mesh2.visual.vertex_colors = cmap2.clip(0, 255)[:len(mesh2.vertices)]
scene.add_geometry(mesh2)

scene.show()

In [69]:
np.sum(np.abs(cmap[template_smpl_corr]/255 - cmap2[shape_template_corr]/255))

128.4549019607843

In [70]:
np.sum(np.abs(cmap[shape_smpl_corr]/255 - cmap2/255))

6063.470588235294

In [72]:
import my_code.datasets.preprocessing as preprocessing
import torch

verts_orig = preprocessing.center(
    torch.tensor(mesh1.vertices)
    )[0]

verts_remeshed = preprocessing.center(
    torch.tensor(mesh2.vertices)
    )[0]

verts_remeshed = preprocessing.scale(
    input_verts=verts_remeshed,
    input_faces=torch.tensor(mesh2.faces),
    ref_verts=verts_orig,
    ref_faces=torch.tensor(mesh1.faces)
)[0]


In [81]:
verts_orig[template_smpl_corr].min(), verts_orig[template_smpl_corr].max()

(tensor(-1.1180, dtype=torch.float64), tensor(1.1180, dtype=torch.float64))

In [84]:
np.mean(np.abs(verts_orig[template_smpl_corr].numpy() - verts_remeshed[shape_template_corr].numpy()))

0.004972067659011989

# Correspondences on FAUST A

In [94]:
faust_a_path = '/home/s94zalek_hpc/shape_matching/data/FAUST_a/off/tr_reg_081.off'
mesh_a = trimesh.load(faust_a_path, process=False)

smpl_path = '/home/s94zalek_hpc/shape_matching/data/FAUST_original/off/tr_reg_001.off'
mesh_smpl = trimesh.load(smpl_path, process=False)

corr_path = '/home/s94zalek_hpc/shape_matching/data/FAUST_a/corres/tr_reg_081.vts'
corr = np.loadtxt(corr_path).astype(int) - 1

In [ ]:
scene.geometry.clear()

# imterpolate the color diagonally based on 0 and 2 coordinate
cmap = trimesh.visual.color.interpolate(
    mesh_a.vertices[:, 0] / mesh_a.vertices[:, 0].max() +\
    mesh_a.vertices[:, 1] / mesh_a.vertices[:, 1].max(),
    'jet')

# add the first mesh
mesh_a.visual.vertex_colors = cmap[:len(mesh1.vertices)].clip(0, 255)
scene.add_geometry(mesh_a)

mesh_smpl.vertices += [1, 0, 0]
mesh_smpl.visual.vertex_colors = cmap[corr].clip(0, 255)
scene.add_geometry(mesh_smpl)

scene.show()